<a href="https://colab.research.google.com/github/Francofus/IMSE441/blob/main/IMSE710.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
from scipy.sparse.csgraph import minimum_spanning_tree
import networkx as nx

In [40]:
# Load the distance matrix from the CSV file
file_path = '/content/drive/MyDrive/IMSE_441/IMSE 710 TSP Data.csv'
distance_matrix_df = pd.read_csv(file_path)

# Set the first column as the index to properly format the DataFrame
distance_matrix_df.set_index(distance_matrix_df.columns[0], inplace=True)

# Print the DataFrame to display the distance matrix with city name
distance_matrix_df
# Confirm the matrix is square
if distance_matrix_df.shape[0] != distance_matrix_df.shape[1]:
    print("The matrix is not square.")
else:
    print("The matrix is square.")

# Convert the DataFrame to a numpy array for processing
distance_matrix = distance_matrix_df.to_numpy()

# Calculate the Minimum Spanning Tree (MST) using scipy's implementation
mst = minimum_spanning_tree(distance_matrix)
mst_weight = mst.toarray().astype(int)
total_mst_weight = np.sum(mst_weight)

# Output the total MST weight
print("Total MST Weight:", total_mst_weight)

# Find the edges that make up the MST
rows, cols = np.where(mst_weight > 0)
cities = distance_matrix_df.index.tolist()
connected_cities = [(cities[row], cities[col], mst_weight[row, col]) for row, col in zip(rows, cols)]

# Output the connected cities by the MST and their distances
for city1, city2, weight in connected_cities:
    print(f"{city1} -- {city2}: {weight}")

The matrix is square.
Total MST Weight: 12452
Atlanta, GA -- Knoxville, TN: 340
Atlanta, GA -- Orlando, FL: 705
Chicago, IL -- Cleveland, OH: 552
Chicago, IL -- Milwaukee, WI: 147
Chicago, IL -- St. Louis, MO: 475
Cleveland, OH -- Pittsburgh, PA: 216
Denver, CO -- Kansas City, KS: 956
Denver, CO -- Salt Lake City, UT: 851
Greensboro, NC -- Roanoke, VA: 160
Greensboro, NC -- Virginia Beach, VA: 406
Houston, TX -- Little Rock, AR: 718
Kansas City, KS -- St. Louis, MO: 401
Knoxville, TN -- Roanoke, VA: 414
Las Vegas, NV -- Salt Lake City, UT: 680
Las Vegas, NV -- San Francisco, CA: 905
Little Rock, AR -- Memphis, TN: 219
Memphis, TN -- St. Louis, MO: 454
Miami, FL -- Orlando, FL: 377
Milwaukee, WI -- Minneapolis, MN: 537
Newark, NJ -- Portland, ME: 524
Newark, NJ -- Virginia Beach, VA: 563
Pittsburgh, PA -- Roanoke, VA: 556
San Francisco, CA -- Seattle, WA: 1296


In [41]:
# Calculate rows and cols from the MST weights
rows, cols = np.where(mst_weight > 0)

# Using a high integer value for exclusion instead of infinity
max_possible_value = np.max(distance_matrix) + 100000  # Set a high value based on the max distance

# Re-examine and filter out MST edges correctly
non_mst_arcs_filtered = distance_matrix.copy()

# Set edges included in the MST and diagonal to the high integer value to effectively exclude them
np.fill_diagonal(non_mst_arcs_filtered, max_possible_value)  # Exclude self-loops
for row, col in zip(rows, cols):
    non_mst_arcs_filtered[row][col] = max_possible_value
    non_mst_arcs_filtered[col][row] = max_possible_value  # Ensure symmetry if necessary

# Find the smallest value in the adjusted matrix which is not part of the MST
min_non_mst_value_correct = np.min(non_mst_arcs_filtered)
min_non_mst_indices_correct = np.where(non_mst_arcs_filtered == min_non_mst_value_correct)

# Correctly identifying the city pairs for the smallest non-MST edge
min_non_mst_cities_final = [(cities[i], cities[j]) for i, j in zip(min_non_mst_indices_correct[0], min_non_mst_indices_correct[1])]

min_non_mst_value_correct, min_non_mst_cities_final


(456,
 [('Greensboro, NC', 'Knoxville, TN'), ('Knoxville, TN', 'Greensboro, NC')])

In [44]:
# Get the list of cities from the DataFrame index
cities = distance_matrix_df.index.tolist()

# Starting city: Kansas City, KS
start_city = 'Kansas City, KS'
start_index = cities.index(start_city)

# Initialize tour and visited set
tour = [start_city]
visited = set([start_index])
current_index = start_index
total_distance = 0  # Initialize total distance
tour_steps = []  # To store each step with distances

# Implementing Nearest Neighbor Heuristic
while len(visited) < len(cities):
    distances = np.array([distance_matrix[current_index][i] if i not in visited else float('inf') for i in range(len(cities))])
    next_city_index = np.argmin(distances)
    next_city = cities[next_city_index]
    tour.append(next_city)
    visited.add(next_city_index)
    tour_distance = distances[next_city_index]
    total_distance += tour_distance
    tour_steps.append({"From": cities[current_index], "To": next_city, "Distance": tour_distance})
    current_index = next_city_index

# Add the return to the starting city
tour.append(start_city)
return_distance = distance_matrix[current_index][start_index]
total_distance += return_distance
tour_steps.append({"From": cities[current_index], "To": start_city, "Distance": return_distance})

# Create a DataFrame from tour steps
tour_df = pd.DataFrame(tour_steps)

# Print the DataFrame and total distance
print(tour_df)
print("Total Distance:", total_distance)

                  From                  To  Distance
0      Kansas City, KS       St. Louis, MO     401.0
1        St. Louis, MO         Memphis, TN     454.0
2          Memphis, TN     Little Rock, AR     219.0
3      Little Rock, AR         Houston, TX     718.0
4          Houston, TX         Atlanta, GA    1268.0
5          Atlanta, GA       Knoxville, TN     340.0
6        Knoxville, TN         Roanoke, VA     414.0
7          Roanoke, VA      Greensboro, NC     160.0
8       Greensboro, NC  Virginia Beach, VA     406.0
9   Virginia Beach, VA          Newark, NJ     563.0
10          Newark, NJ        Portland, ME     524.0
11        Portland, ME      Pittsburgh, PA    1072.0
12      Pittsburgh, PA       Cleveland, OH     216.0
13       Cleveland, OH         Chicago, IL     552.0
14         Chicago, IL       Milwaukee, WI     147.0
15       Milwaukee, WI     Minneapolis, MN     537.0
16     Minneapolis, MN          Denver, CO    1460.0
17          Denver, CO  Salt Lake City, UT    

In [45]:
# Define the tour cities in order
tour_cities = [
    "Kansas City, KS", "St. Louis, MO", "Memphis, TN", "Little Rock, AR", "Houston, TX",
    "Miami, FL", "Orlando, FL", "Atlanta, GA", "Knoxville, TN", "Roanoke, VA",
    "Greensboro, NC", "Virginia Beach, VA", "Newark, NJ", "Portland, ME", "Pittsburgh, PA",
    "Cleveland, OH", "Chicago, IL", "Milwaukee, WI", "Minneapolis, MN", "Denver, CO",
    "Salt Lake City, UT", "Seattle, WA", "San Francisco, CA", "Las Vegas, NV", "Kansas City, KS"
]

# Map the city names to their indices based on the DataFrame index
tour_indices = [cities.index(city) for city in tour_cities]

# Calculate the total distance of the tour
total_tour_distance = 0
for i in range(len(tour_indices) - 1):
    start = tour_indices[i]
    end = tour_indices[i + 1]
    distance = distance_matrix[start][end]
    total_tour_distance += distance

# Print the total distance of the tour
total_tour_distance


17719

In [51]:
# Start at Kansas City, KS
start_city = 'Kansas City, KS'
start_index = cities.index(start_city)

# Initialize tour
tour = [start_index]
unvisited = set(range(len(cities))) - {start_index}

# Find the farthest city from the start city
farthest = max(unvisited, key=lambda x: distance_matrix[start_index][x])
tour.append(farthest)
unvisited.remove(farthest)

# Continue with Farthest Insertion
while unvisited:
    next_city, max_dist = max(((city, max(distance_matrix[city][i] for i in tour)) for city in unvisited), key=lambda x: x[1])
    # Insert at the best position
    best_pos, min_increase = min(((i, distance_matrix[tour[i-1]][next_city] + distance_matrix[next_city][tour[i]] - distance_matrix[tour[i-1]][tour[i]]) for i in range(1, len(tour))), key=lambda x: x[1])
    tour.insert(best_pos, next_city)
    unvisited.remove(next_city)

# Close the tour
tour.append(start_index)

# Calculate total distance and collect data for the table
tour_steps = []
total_distance = 0
for i in range(len(tour) - 1):
    from_city = cities[tour[i]]
    to_city = cities[tour[i+1]]
    segment_distance = distance_matrix[tour[i]][tour[i+1]]
    total_distance += segment_distance
    tour_steps.append({'From': from_city, 'To': to_city, 'Distance': segment_distance})

# Create a DataFrame from tour steps
tour_df = pd.DataFrame(tour_steps)
tour_df.loc['Total'] = pd.Series({'Distance': total_distance}, index=['Distance'])

# Display the table
print(tour_df)

                     From                  To  Distance
0         Kansas City, KS         Houston, TX    1265.0
1             Houston, TX     Little Rock, AR     718.0
2         Little Rock, AR         Memphis, TN     219.0
3             Memphis, TN       Knoxville, TN     627.0
4           Knoxville, TN         Atlanta, GA     340.0
5             Atlanta, GA         Orlando, FL     705.0
6             Orlando, FL           Miami, FL     377.0
7               Miami, FL      Greensboro, NC    1270.0
8          Greensboro, NC         Roanoke, VA     160.0
9             Roanoke, VA  Virginia Beach, VA     470.0
10     Virginia Beach, VA          Newark, NJ     563.0
11             Newark, NJ        Portland, ME     524.0
12           Portland, ME      Pittsburgh, PA    1072.0
13         Pittsburgh, PA       Cleveland, OH     216.0
14          Cleveland, OH         Chicago, IL     552.0
15            Chicago, IL       Milwaukee, WI     147.0
16          Milwaukee, WI     Minneapolis, MN   

In [53]:
tour_cities = [
    'Kansas City, KS', 'Houston, TX', 'Little Rock, AR', 'Memphis, TN', 'St. Louis, MO',
    'Knoxville, TN', 'Atlanta, GA', 'Orlando, FL', 'Miami, FL', 'Greensboro, NC',
    'Roanoke, VA', 'Virginia Beach, VA', 'Newark, NJ', 'Portland, ME', 'Pittsburgh, PA',
    'Cleveland, OH', 'Chicago, IL', 'Milwaukee, WI', 'Minneapolis, MN', 'Seattle, WA',
    'San Francisco, CA', 'Las Vegas, NV', 'Salt Lake City, UT', 'Denver, CO', 'Kansas City, KS'
]

# Calculate the total distance of the tour
total_distance = 0
tour_steps = []

for i in range(len(tour_cities) - 1):
    from_city = tour_cities[i]
    to_city = tour_cities[i + 1]
    distance = distance_matrix_df.at[from_city, to_city]
    total_distance += distance
    tour_steps.append({'From': from_city, 'To': to_city, 'Distance': distance})

# Create a DataFrame from tour steps
tour_df = pd.DataFrame(tour_steps)
tour_df.loc['Total'] = pd.Series({'Distance': total_distance}, index=['Distance'])

# Display the table
print(tour_df)

                     From                  To  Distance
0         Kansas City, KS         Houston, TX    1265.0
1             Houston, TX     Little Rock, AR     718.0
2         Little Rock, AR         Memphis, TN     219.0
3             Memphis, TN       St. Louis, MO     454.0
4           St. Louis, MO       Knoxville, TN     777.0
5           Knoxville, TN         Atlanta, GA     340.0
6             Atlanta, GA         Orlando, FL     705.0
7             Orlando, FL           Miami, FL     377.0
8               Miami, FL      Greensboro, NC    1270.0
9          Greensboro, NC         Roanoke, VA     160.0
10            Roanoke, VA  Virginia Beach, VA     470.0
11     Virginia Beach, VA          Newark, NJ     563.0
12             Newark, NJ        Portland, ME     524.0
13           Portland, ME      Pittsburgh, PA    1072.0
14         Pittsburgh, PA       Cleveland, OH     216.0
15          Cleveland, OH         Chicago, IL     552.0
16            Chicago, IL       Milwaukee, WI   